# Database Query Notebook

PostgreSQL 데이터베이스에서 뉴스 기사를 조회하는 노트북입니다.

In [13]:
!pip install psycopg2-binary pandas

In [14]:
import psycopg2
import pandas as pd
from datetime import datetime

# PostgreSQL 연결 설정
def get_db_connection():
    return psycopg2.connect(
        host="localhost",  # Docker service name
        database="news_db",
        user="postgres",
        password="password"
    )

In [15]:
# 접속 테스트

import psycopg2

conn = psycopg2.connect(
    host='postgres',
    port=5432,
    database='news_db',
    user='postgres',
    password='password'
)

cur = conn.cursor()
cur.execute("SELECT * FROM articles LIMIT 5;")
rows = cur.fetchall()
for row in rows:
    print(row)

cur.close()
conn.close()


(1, '카카오', 'https://www.widedaily.com/news/articleView.html?idxno=253099', '카카오프렌즈, 베어브릭 컬래버레이션 굿즈 출시', 'https://www.widedaily.com/news/articleView.html?idxno=253099', '카카오가 카카오프렌즈와 베어브릭이 협업한 컬래버레이션 굿즈를 출시했다고 5일 밝혔다. 카카오는 지난 2021년 베어브릭 제조사 메디콤토이와 제휴해 황금 라이언 베어브릭을 선보인 바 있다. 출시 당일 카카오프렌즈...', '', 'www.widedaily.com', datetime.datetime(2024, 12, 5, 2, 16, tzinfo=datetime.timezone.utc), '2024.12.05', datetime.datetime(2024, 12, 5, 2, 17, 4, tzinfo=datetime.timezone.utc), False, True, True)
(2, '카카오모빌리티', 'http://www.smedaily.co.kr/news/articleView.html?idxno=311556', '카카오 주가 20일새 40% 오른 이유', 'http://www.smedaily.co.kr/news/articleView.html?idxno=311556', '삼성증권 오동환 연구원은 "(카카오는) 컬러버스, 세나테크놀로지 등 비주력 손자회사 매각을 진행 중이나 워낙 커진 조직 규모 탓에 재무 영향은 제한적"이라며 "기존 모빌리티, 페이 등을 이을 수 있는 혁신적인 신규...', '중소기업신문', 'www.smedaily.co.kr', datetime.datetime(2024, 12, 5, 1, 26, tzinfo=datetime.timezone.utc), '2024.12.05', datetime.datetime(2024, 12, 5, 2, 16, 17, tzinfo=datetime.timezone.utc), False, True, True)
(3, '카카오

## 날짜별 기사 검색

In [16]:
def count_by_date():
    """
    특정 날짜의 기사를 검색합니다.
    :param date_str: 'YYYY-MM-DD' 형식의 날짜 문자열
    """
    conn = get_db_connection()
    
    query = """
    SELECT 
    DATE(published_at) AS date,
    COUNT(*) AS total_articles,
    SUM(CASE WHEN is_naver_news IS True THEN 1 ELSE 0 END) AS naver_articles
    FROM articles
    GROUP BY DATE(published_at)
    ORDER BY date DESC;
    """
    
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

# 사용 예시:
df = count_by_date()
display(df)

OperationalError: connection to server at "localhost" (::1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?


In [17]:
def search_by_date(date_str):
    """
    특정 날짜의 기사를 검색합니다.
    :param date_str: 'YYYY-MM-DD' 형식의 날짜 문자열
    """
    conn = get_db_connection()
    
    query = """
    SELECT 
        id,
        title,
        publisher,
        naver_link,
        published_at
    FROM articles
    WHERE DATE(published_at) = %s
    ORDER BY published_at DESC
    """
    
    df = pd.read_sql_query(query, conn, params=[date_str])
    conn.close()
    return df

# 사용 예시:
df = search_by_date('2024-10-15')
display(df)

OperationalError: connection to server at "localhost" (::1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?


## 키워드로 기사 검색

In [18]:
def search_by_keyword(keyword):
    """
    키워드가 포함된 기사를 검색합니다.
    :param keyword: 검색할 키워드
    """
    conn = get_db_connection()
    
    query = """
    SELECT 
        id,
        title,
        publisher,
        naver_link,
        published_at
    FROM articles
    WHERE 
        title LIKE %s OR
        main_keyword LIKE %s
    ORDER BY published_at DESC
    """
    
    search_pattern = f'%{keyword}%'
    df = pd.read_sql_query(query, conn, params=[search_pattern, search_pattern])
    conn.close()
    return df

# 사용 예시:
df = search_by_keyword('카카오모빌리티')
display(df)

OperationalError: connection to server at "localhost" (::1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?


## 날짜와 키워드로 기사 검색

In [4]:
def search_by_date_and_keyword(date_str, keyword):
    """
    특정 날짜와 키워드로 기사를 검색합니다.
    :param date_str: 'YYYY-MM-DD' 형식의 날짜 문자열
    :param keyword: 검색할 키워드
    """
    conn = get_db_connection()
    
    query = """
    SELECT 
        id,
        title,
        publisher,
        naver_link,
        published_at
    FROM articles
    WHERE 
        DATE(published_at) = %s AND
        (title LIKE %s OR main_keyword LIKE %s)
    ORDER BY published_at DESC
    """
    
    search_pattern = f'%{keyword}%'
    df = pd.read_sql_query(query, conn, params=[date_str, search_pattern, search_pattern])
    conn.close()
    return df

# 사용 예시:
df = search_by_date_and_keyword('2024-10-15', '카카오모빌리티')
display(df)

/var/folders/m3/j7b0h30d7412rbfrmrz8rqqc0000gn/T/ipykernel_29290/3369546302.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn, params=[date_str, search_pattern, search_pattern])


,id,title,publisher,naver_link,published_at


## 기사 내용 조회

In [22]:
def get_article_content(article_id):
    """
    특정 기사의 내용을 조회합니다.
    :param article_id: 기사 ID
    """
    conn = get_db_connection()
    
    query = """
    SELECT 
        a.id,
        a.title,
        a.publisher,
        c.content,
        a.published_at
    FROM articles a
    LEFT JOIN contents c ON a.id = c.article_id
    WHERE a.id = %s
    """
    
    df = pd.read_sql_query(query, conn, params=[article_id])
    conn.close()
    return df

# 사용 예시:
df = get_article_content(3514)
display(df)

/var/folders/m3/j7b0h30d7412rbfrmrz8rqqc0000gn/T/ipykernel_37724/408960758.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn, params=[article_id])


,id,title,publisher,content,published_at


## 카카오모빌리티 관련 기사 검색 예시

In [25]:
# 특정 날짜의 카카오모빌리티 관련 기사 검색
date_str = '2024-09-01'  # 날짜를 원하는 날짜로 변경하세요
keyword = '카카오모빌리티'

df = search_by_date_and_keyword(date_str, keyword)
display(df)

OperationalError: connection to server at "localhost" (::1), port 5432 failed: FATAL:  database "news_db" does not exist


In [9]:
# 특정 날짜의 카카오모빌리티 관련 기사 검색
date_str = '2024-10-01'  # 날짜를 원하는 날짜로 변경하세요
keyword = '카카오모빌리티'

df = search_by_date_and_keyword(date_str, keyword)
display(df)

/var/folders/m3/j7b0h30d7412rbfrmrz8rqqc0000gn/T/ipykernel_17633/1654268631.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn, params=[date_str, search_pattern, search_pattern])


,id,title,publisher,naver_link,published_at
0,49,"공정위,카카오모빌리티과징금 724억 부과한 이유는?",서울타임즈뉴스,https://www.seoultimes.news/news/article.html?...,2024-10-01 15:32:00+00:00
1,51,“영업비밀 요구·‘콜 차단’까지”…카카오모빌리티과징금 724억,KBS,https://n.news.naver.com/mnews/article/056/001...,2024-10-01 15:32:00+00:00
2,53,영업비밀 안 주면 '콜 차단'...카카오에 과징금 724억 원·검찰 고발,YTN,https://n.news.naver.com/mnews/article/052/000...,2024-10-01 15:32:00+00:00
3,55,“영업비밀 요구·‘콜 차단’까지”…카카오모빌리티과징금 724억,KBS,https://n.news.naver.com/mnews/article/056/001...,2024-10-01 15:32:00+00:00
4,57,영업기밀 안 주자 '콜 차단'…카카오T에 과징금 724억 원,SBS,https://n.news.naver.com/mnews/article/055/000...,2024-10-01 15:32:00+00:00
...,...,...,...,...,...
279,651,[2024 국정감사 기획] 22대 첫 국감도 기업인 줄소환,대한경제,https://www.dnews.co.kr/uhtml/view.jsp?idxno=2...,2024-10-01 15:32:00+00:00
280,653,주요 기업 CEO 국감 줄소환… 확정 명단·사유 보니,조선비즈,https://n.news.naver.com/mnews/article/366/000...,2024-10-01 15:32:00+00:00
281,655,불황 넘기 바쁜데… “또 망신주기 국감 불려나가” 재계 한숨,국민일보,https://n.news.naver.com/mnews/article/005/000...,2024-10-01 15:32:00+00:00
282,657,"한국관광공사,카카오모빌리티와 손잡고 단풍 여행지 소개",여행신문,https://www.traveltimes.co.kr/news/articleView...,2024-10-01 15:32:00+00:00


In [ ]:
def search_by_date_and_keyword_all(date_str, keyword):
    """
    특정 날짜와 키워드로 기사를 검색합니다.
    :param date_str: 'YYYY-MM-DD' 형식의 날짜 문자열
    :param keyword: 검색할 키워드
    """
    conn = get_db_connection()
    
    query = """
    SELECT 
        *
    FROM articles
    WHERE 
        DATE(published_at) = %s AND
        (title LIKE %s OR main_keyword LIKE %s)
    ORDER BY published_at DESC
    """
    
    search_pattern = f'%{keyword}%'
    df = pd.read_sql_query(query, conn, params=[date_str, search_pattern, search_pattern])
    conn.close()
    return df

# 사용 예시:
df = search_by_date_and_keyword_all('2024-10-01', '카카오모빌리티')
display(df)


In [23]:
def get_all_comments(limit=50):
    """
    모든 댓글을 collected_at 기준으로 내림차순 정렬하여 조회합니다.
    
    :param limit: 조회할 댓글의 수 (기본값: 50)
    :return: pandas DataFrame 형식의 댓글 데이터
    """
    conn = get_db_connection()
    
    query = """
    SELECT 
        *
    FROM comments
    ORDER BY collected_at DESC
    LIMIT %s;
    """
    
    df = pd.read_sql_query(query, conn, params=[limit])
    conn.close()
    return df

# 사용 예시:
df_all_comments = get_all_comments()
display(df_all_comments)

/var/folders/m3/j7b0h30d7412rbfrmrz8rqqc0000gn/T/ipykernel_37724/1729070522.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn, params=[limit])


,id,article_id,comment_no,parent_comment_no,content,username,profile_url,timestamp,collected_at,likes,dislikes,reply_count,is_reply,is_deleted,delete_type
0,5937,3102,844493191053836396,844493191053836396,"모든 국민들이 다 관용이 없다면 그 또한 깨끗한 사회가 될 수도 있는데, 이런 관용...",dowh****,https://phinf.pstatic.net/contact/0/2011/4/5/d...,2024-12-02 23:31:32+00:00,2024-12-03 01:33:56.233391+00:00,0,4,0,False,False,None
1,5936,3102,844494886844498114,844494886844498114,"사위왈,장모는 남에게 10원도 피해 준 적 없다: 약정서 위조해 동업자 정대택이 감...",shs5****,https://static.nid.naver.com/images/web/user/d...,2024-12-02 23:57:52+00:00,2024-12-03 01:33:56.232965+00:00,0,2,0,False,False,None
2,5935,3102,844495418665467930,844495418665467930,찌라시 언론이 영웅되는 오늘의 한국정치!,nnn0****,https://phinf.pstatic.net/contact/20231129_82/...,2024-12-03 00:06:07+00:00,2024-12-03 01:33:56.232533+00:00,0,2,1,False,False,None
3,5934,3102,844494988329877596,844494988329877596,비교할껄 비교해라. 주가조작.분식회계등 본질이 사기꾼범죄와 비교하나? 미국은 분식회...,meic****,https://static.nid.naver.com/images/web/user/d...,2024-12-02 23:59:26+00:00,2024-12-03 01:33:56.231901+00:00,0,1,0,False,False,None
4,5933,3102,844493749533802560,844493749533802560,멀리 볼수있는 혜안을 가집시다 가족끼리 미주알 고주알 들추다 남의집 머...,agaa****,https://static.nid.naver.com/images/web/user/d...,2024-12-02 23:40:13+00:00,2024-12-03 01:33:56.231109+00:00,0,0,0,False,False,None
5,5932,3102,844495176184365145,844495176184365145,영웅을죽이는 조선은?,hyun****,https://static.nid.naver.com/images/web/user/d...,2024-12-03 00:02:21+00:00,2024-12-03 01:33:56.229561+00:00,0,0,0,False,False,None
6,5931,3102,844495230827757624,844495230827757624,상법개정안이나 통과시키자. 그럼 영웅 욕할일도 없어질듯.,cjeu****,https://static.nid.naver.com/images/web/user/d...,2024-12-03 00:03:12+00:00,2024-12-03 01:33:56.229130+00:00,0,0,0,False,False,None
7,5930,3102,844495323068891286,844495323068891286,기업인이 국가경제에 기여하는 바가 매우 큰 것이 분명하다. 그러나 비뚤어진 기업가정...,lts0****,https://phinf.pstatic.net/contact/20211210_249...,2024-12-03 00:04:38+00:00,2024-12-03 01:33:56.228693+00:00,0,0,0,False,False,None
8,5929,3102,844495462353338572,844495462353338572,시기와 질투가 많은 민족성. 사돈이 땅 사면 배아프지,park****,https://static.nid.naver.com/images/web/user/d...,2024-12-03 00:06:48+00:00,2024-12-03 01:33:56.228269+00:00,0,0,0,False,False,None
9,5928,3102,844499421440770147,844499421440770147,이런기업이 크게성장할수있는 풍토를 만들어 우리도 선진국으로 도약하수있습니다.,bhs4****,https://phinf.pstatic.net/contact/20221005_45/...,2024-12-03 01:08:15+00:00,2024-12-03 01:33:56.227468+00:00,0,0,0,False,False,None
